In [1]:
from dotenv import load_dotenv
load_dotenv()  # loads variables from .env file

import os
print(os.getenv("LANGCHAIN_API_KEY"))  # test: should print your key (or None if not set)

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


In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

In [3]:
chunks = []
for chunk in model.stream("what color is the sky?"):
    chunks.append(chunk)
    print(chunk.content, end="|", flush=True)

|The| color| of| the| sky| can| vary| depending| on| several| factors|,| including| the| time| of| day|,| weather| conditions|,| and| atmospheric| particles|.| During| a| clear| day|,| the| sky| typically| appears| blue| due| to| the| scattering| of| sunlight| by| the| atmosphere|.| At| sunrise| and| sunset|,| the| sky| can| display| a| range| of| colors| such| as| orange|,| pink|,| and| red| because| the| sunlight| passes| through| more| of| the| Earth's| atmosphere|,| scattering| shorter| wavelengths| and| allowing| longer| wavelengths| to| dominate|.| On| cloudy| days|,| the| sky| may| appear| gray| or| white|,| and| during| storm|y| weather|,| it| can| take| on| darker| hues|.||

In [4]:
chunks = []
async for chunk in model.astream("what color is the sky?"):
    chunks.append(chunk)
    print(chunk.content, end="|", flush=True)

|The| sky| typically| appears| blue| during| the| day| due| to| the| scattering| of| sunlight| by| the| Earth's| atmosphere|.| This| phenomenon|,| known| as| Ray|leigh| scattering|,| causes| shorter| blue| wavelengths| of| light| to| scatter| more| than| other| colors|,| making| the| sky| look| predominantly| blue|.| At| sunrise| and| sunset|,| the| sky| can| display| a| variety| of| colors|,| including| reds|,| oranges|,| and| pur|ples|,| as| the| sunlight| passes| through| more| of| the| atmosphere|.| At| night|,| the| sky| appears| black| or| dark| blue|,| punct|uated| by| stars| and| celestial| objects|.| Weather| conditions| can| also| affect| the| appearance| of| the| sky|,| leading| to| gray|,| over|cast| skies|,| or| vibrant| hues| during| specific| atmospheric| conditions|.||

In [5]:
chunks[1]

AIMessageChunk(content='The', additional_kwargs={}, response_metadata={}, id='run--b6941230-9c61-4238-9686-dddd0b189ceb')

In [6]:
chunks[0] + chunks[1] + chunks[2] + chunks[3] + chunks[4] + chunks[5]

AIMessageChunk(content='The sky typically appears blue', additional_kwargs={}, response_metadata={}, id='run--b6941230-9c61-4238-9686-dddd0b189ceb')

#### Chains

In [9]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
parser = StrOutputParser()
chain = prompt | model | parser

for chunk in chain.stream({"topic": "parrot"}):
    print(chunk, end="|", flush=True)

|Why| did| the| par|rot| wear| a| rain|coat|?

|Because| it| wanted| to| be| a| poly|-uns|aturated| bird|!||

In [10]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
parser = StrOutputParser()
chain = prompt | model | parser

async for chunk in chain.astream({"topic": "parrot"}):
    print(chunk, end="|", flush=True)

|Why| did| the| par|rot| wear| a| rain|coat|?

|Because| it| wanted| to| be| a| poly| uns|aturated|!||

In [12]:
from langchain_core.output_parsers import JsonOutputParser
chain = (
    model | JsonOutputParser()
)

async for text in chain.astream(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`"
):
    print(text, flush=True)

{}
{'countries': []}
{'countries': [{}]}
{'countries': [{'name': ''}]}
{'countries': [{'name': 'France'}]}
{'countries': [{'name': 'France', 'population': 652}]}
{'countries': [{'name': 'France', 'population': 652735}]}
{'countries': [{'name': 'France', 'population': 65273511}]}
{'countries': [{'name': 'France', 'population': 65273511}, {}]}
{'countries': [{'name': 'France', 'population': 65273511}, {'name': ''}]}
{'countries': [{'name': 'France', 'population': 65273511}, {'name': 'Spain'}]}
{'countries': [{'name': 'France', 'population': 65273511}, {'name': 'Spain', 'population': 467}]}
{'countries': [{'name': 'France', 'population': 65273511}, {'name': 'Spain', 'population': 467547}]}
{'countries': [{'name': 'France', 'population': 65273511}, {'name': 'Spain', 'population': 46754778}]}
{'countries': [{'name': 'France', 'population': 65273511}, {'name': 'Spain', 'population': 46754778}, {}]}
{'countries': [{'name': 'France', 'population': 65273511}, {'name': 'Spain', 'population': 467

In [13]:
from langchain_core.output_parsers import (
    JsonOutputParser,
)


# A function that operates on finalized inputs
# rather than on an input_stream
def _extract_country_names(inputs):
    """A function that does not operates on input streams and breaks streaming."""
    if not isinstance(inputs, dict):
        return ""

    if "countries" not in inputs:
        return ""

    countries = inputs["countries"]

    if not isinstance(countries, list):
        return ""

    country_names = [
        country.get("name") for country in countries if isinstance(country, dict)
    ]
    return country_names


chain = model | JsonOutputParser() | _extract_country_names

async for text in chain.astream(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`"
):
    print(text, end="|", flush=True)

['France', 'Spain', 'Japan']|

In [14]:
from langchain_core.output_parsers import JsonOutputParser


async def _extract_country_names_streaming(input_stream):
    """A function that operates on input streams."""
    country_names_so_far = set()

    async for input in input_stream:
        if not isinstance(input, dict):
            continue

        if "countries" not in input:
            continue

        countries = input["countries"]

        if not isinstance(countries, list):
            continue

        for country in countries:
            name = country.get("name")
            if not name:
                continue
            if name not in country_names_so_far:
                yield name
                country_names_so_far.add(name)


chain = model | JsonOutputParser() | _extract_country_names_streaming

async for text in chain.astream(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`",
):
    print(text, end="|", flush=True)

France|Spain|Japan|

In [24]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho", "harrison likes spicy food"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

chunks = [chunk for chunk in retriever.stream("where did harrison work?")]
chunks

[[Document(id='acc6027c-68cb-42ad-8302-77eb0e8d9b4a', metadata={}, page_content='harrison worked at kensho'),
  Document(id='1e85d5d6-c9ba-4871-b6c6-5811545f4648', metadata={}, page_content='harrison likes spicy food')]]

In [25]:
retrieval_chain = (
    {
        "context": retriever.with_config(run_name="Docs"),
        "question": RunnablePassthrough(),
    }
    | prompt
    | model
    | StrOutputParser()
)

In [26]:
for chunk in retrieval_chain.stream(
    "Where did harrison work? " "Write 3 made up sentences about this place."
):
    print(chunk, end="|", flush=True)

|H|arrison| worked| at| Kens|ho|.| Kens|ho| is| known| for| its| innovative| approach| to| data| analytics|,| utilizing| cutting|-edge| artificial| intelligence| techniques| to| provide| insights| for| businesses|.| The| atmosphere| at| Kens|ho| is| vibrant| and| collaborative|,| with| team| members| often| brainstorming| in| open| spaces| filled| with| greenery|.| Employee| wellness| is| a| priority|,| and| the| company| regularly| organizes| fun| team|-building| activities| that| include| everything| from| cooking| classes| to| outdoor| adventures|.||

In [27]:
import langchain_core

langchain_core.__version__

'0.3.71'